# 📈 Linear Regression - Complete Guide

This notebook provides a comprehensive introduction to Linear Regression, covering:
- Mathematical foundation
- Implementation from scratch
- Scikit-learn implementation
- Model evaluation and interpretation
- Assumptions checking

---

## 📚 1. Import Libraries and Setup

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Statistical analysis
import scipy.stats as stats
from scipy import stats

# Visualization settings
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!")

## 🧮 2. Mathematical Foundation

### Simple Linear Regression
$$y = \beta_0 + \beta_1 x + \epsilon$$

### Multiple Linear Regression
$$y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + ... + \beta_n x_n + \epsilon$$

### Normal Equation
$$\boldsymbol{\beta} = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}$$

### Cost Function (MSE)
$$J(\boldsymbol{\beta}) = \frac{1}{2m} \sum_{i=1}^{m}(h_{\boldsymbol{\beta}}(x^{(i)}) - y^{(i)})^2$$

## 📊 3. Generate Sample Data

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Generate sample data
n_samples = 100
X = np.random.randn(n_samples, 1) * 2
y = 3 + 2 * X.ravel() + np.random.randn(n_samples) * 0.5

# Create DataFrame for easier handling
df = pd.DataFrame({
    'feature': X.ravel(),
    'target': y
})

print(f"Dataset shape: {df.shape}")
print(f"\nFirst 5 rows:")
print(df.head())

# Visualize the data
plt.figure(figsize=(10, 6))
plt.scatter(X, y, alpha=0.6, color='blue')
plt.xlabel('Feature (X)')
plt.ylabel('Target (y)')
plt.title('Sample Dataset for Linear Regression')
plt.grid(True, alpha=0.3)
plt.show()

## 🔧 4. Linear Regression from Scratch

In [ ]:
class LinearRegressionScratch:
    """
    Linear Regression implementation from scratch using Normal Equation
    """
    
    def __init__(self):
        self.coefficients = None
        self.intercept = None
    
    def fit(self, X, y):
        """
        Fit the linear regression model using Normal Equation
        """
        # Add bias column (intercept term)
        X_with_bias = np.c_[np.ones(X.shape[0]), X]
        
        # Normal equation: β = (X^T X)^(-1) X^T y
        XtX = X_with_bias.T @ X_with_bias
        Xty = X_with_bias.T @ y
        
        # Solve for coefficients
        beta = np.linalg.solve(XtX, Xty)
        
        # Store intercept and coefficients
        self.intercept = beta[0]
        self.coefficients = beta[1:]
        
        return self
    
    def predict(self, X):
        """
        Make predictions using the fitted model
        """
        return self.intercept + X @ self.coefficients
    
    def score(self, X, y):
        """
        Calculate R² score
        """
        y_pred = self.predict(X)
        ss_res = np.sum((y - y_pred) ** 2)
        ss_tot = np.sum((y - np.mean(y)) ** 2)
        return 1 - (ss_res / ss_tot)

# Test our implementation
model_scratch = LinearRegressionScratch()
model_scratch.fit(X, y)

print(f"Intercept: {model_scratch.intercept:.4f}")
print(f"Coefficient: {model_scratch.coefficients[0]:.4f}")
print(f"R² Score: {model_scratch.score(X, y):.4f}")

## 🛠️ 5. Scikit-learn Implementation

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Create and fit the model
model_sklearn = LinearRegression()
model_sklearn.fit(X_train, y_train)

# Make predictions
y_train_pred = model_sklearn.predict(X_train)
y_test_pred = model_sklearn.predict(X_test)

print("📊 Model Parameters:")
print(f"Intercept: {model_sklearn.intercept_:.4f}")
print(f"Coefficient: {model_sklearn.coef_[0]:.4f}")

print("\n📈 Performance Metrics:")
print(f"Training R²: {r2_score(y_train, y_train_pred):.4f}")
print(f"Test R²: {r2_score(y_test, y_test_pred):.4f}")
print(f"Training RMSE: {np.sqrt(mean_squared_error(y_train, y_train_pred)):.4f}")
print(f"Test RMSE: {np.sqrt(mean_squared_error(y_test, y_test_pred)):.4f}")
print(f"Test MAE: {mean_absolute_error(y_test, y_test_pred):.4f}")

## 📊 6. Model Visualization

In [ ]:
# Create visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Regression line
axes[0, 0].scatter(X_train, y_train, alpha=0.6, label='Training data', color='blue')
axes[0, 0].scatter(X_test, y_test, alpha=0.6, label='Test data', color='red')

# Plot regression line
X_range = np.linspace(X.min(), X.max(), 100).reshape(-1, 1)
y_range_pred = model_sklearn.predict(X_range)
axes[0, 0].plot(X_range, y_range_pred, color='green', linewidth=2, label='Regression line')

axes[0, 0].set_xlabel('Feature (X)')
axes[0, 0].set_ylabel('Target (y)')
axes[0, 0].set_title('Linear Regression Fit')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Residuals vs Fitted
residuals = y_test - y_test_pred
axes[0, 1].scatter(y_test_pred, residuals, alpha=0.6)
axes[0, 1].axhline(y=0, color='red', linestyle='--')
axes[0, 1].set_xlabel('Fitted Values')
axes[0, 1].set_ylabel('Residuals')
axes[0, 1].set_title('Residuals vs Fitted Values')
axes[0, 1].grid(True, alpha=0.3)

# 3. Q-Q plot for residuals
stats.probplot(residuals, dist="norm", plot=axes[1, 0])
axes[1, 0].set_title('Q-Q Plot of Residuals')
axes[1, 0].grid(True, alpha=0.3)

# 4. Actual vs Predicted
axes[1, 1].scatter(y_test, y_test_pred, alpha=0.6)
axes[1, 1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
                'r--', linewidth=2, label='Perfect prediction')
axes[1, 1].set_xlabel('Actual Values')
axes[1, 1].set_ylabel('Predicted Values')
axes[1, 1].set_title('Actual vs Predicted Values')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## ✅ 7. Assumption Checking

In [ ]:
def check_assumptions(X, y, model, alpha=0.05):
    """
    Check linear regression assumptions
    """
    y_pred = model.predict(X)
    residuals = y - y_pred
    
    print("🔍 LINEAR REGRESSION ASSUMPTIONS CHECK\n")
    print("=" * 50)
    
    # 1. Linearity (correlation between X and y)
    correlation = np.corrcoef(X.ravel(), y)[0, 1]
    print(f"1. LINEARITY")
    print(f"   Correlation coefficient: {correlation:.4f}")
    print(f"   Status: {'✅ GOOD' if abs(correlation) > 0.3 else '⚠️ WEAK LINEAR RELATIONSHIP'}\n")
    
    # 2. Independence (Durbin-Watson test)
    from statsmodels.stats.diagnostic import durbin_watson
    dw_stat = durbin_watson(residuals)
    print(f"2. INDEPENDENCE")
    print(f"   Durbin-Watson statistic: {dw_stat:.4f}")
    print(f"   Status: {'✅ GOOD' if 1.5 < dw_stat < 2.5 else '⚠️ POTENTIAL AUTOCORRELATION'}\n")
    
    # 3. Homoscedasticity (Breusch-Pagan test)
    from statsmodels.stats.diagnostic import het_breuschpagan
    _, bp_pvalue, _, _ = het_breuschpagan(residuals, X)
    print(f"3. HOMOSCEDASTICITY")
    print(f"   Breusch-Pagan test p-value: {bp_pvalue:.4f}")
    print(f"   Status: {'✅ GOOD' if bp_pvalue > alpha else '⚠️ HETEROSCEDASTICITY DETECTED'}\n")
    
    # 4. Normality of residuals (Shapiro-Wilk test)
    _, shapiro_pvalue = stats.shapiro(residuals)
    print(f"4. NORMALITY OF RESIDUALS")
    print(f"   Shapiro-Wilk test p-value: {shapiro_pvalue:.4f}")
    print(f"   Status: {'✅ GOOD' if shapiro_pvalue > alpha else '⚠️ RESIDUALS NOT NORMALLY DISTRIBUTED'}\n")
    
    # 5. No multicollinearity (for multiple features)
    if X.shape[1] > 1:
        from statsmodels.stats.outliers_influence import variance_inflation_factor
        vif_data = pd.DataFrame()
        vif_data["Feature"] = [f"X{i}" for i in range(X.shape[1])]
        vif_data["VIF"] = [variance_inflation_factor(X, i) for i in range(X.shape[1])]
        
        print(f"5. MULTICOLLINEARITY")
        print(vif_data)
        max_vif = vif_data["VIF"].max()
        print(f"   Status: {'✅ GOOD' if max_vif < 5 else '⚠️ HIGH MULTICOLLINEARITY'}\n")
    else:
        print(f"5. MULTICOLLINEARITY")
        print(f"   Single feature - not applicable\n")
    
    print("=" * 50)
    print("✅ = Assumption satisfied")
    print("⚠️ = Assumption violated or needs attention")

# Check assumptions
check_assumptions(X_test, y_test, model_sklearn)

## 🎯 8. Cross-Validation

In [ ]:
# Perform cross-validation
cv_scores = cross_val_score(model_sklearn, X, y, cv=5, scoring='r2')
cv_rmse = -cross_val_score(model_sklearn, X, y, cv=5, scoring='neg_root_mean_squared_error')

print("🔄 Cross-Validation Results:")
print(f"R² Scores: {cv_scores}")
print(f"Mean R²: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
print(f"\nRMSE Scores: {cv_rmse}")
print(f"Mean RMSE: {cv_rmse.mean():.4f} (+/- {cv_rmse.std() * 2:.4f})")

# Visualize cross-validation scores
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].boxplot(cv_scores)
axes[0].set_title('Cross-Validation R² Scores')
axes[0].set_ylabel('R² Score')
axes[0].grid(True, alpha=0.3)

axes[1].boxplot(cv_rmse)
axes[1].set_title('Cross-Validation RMSE Scores')
axes[1].set_ylabel('RMSE')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 📝 9. Key Takeaways and Interpretation

In [ ]:
print("📋 LINEAR REGRESSION ANALYSIS SUMMARY")
print("=" * 50)

print(f"📊 MODEL PERFORMANCE:")
print(f"   • R² Score: {r2_score(y_test, y_test_pred):.4f}")
print(f"   • RMSE: {np.sqrt(mean_squared_error(y_test, y_test_pred)):.4f}")
print(f"   • MAE: {mean_absolute_error(y_test, y_test_pred):.4f}")

print(f"\n🎯 MODEL PARAMETERS:")
print(f"   • Intercept (β₀): {model_sklearn.intercept_:.4f}")
print(f"   • Slope (β₁): {model_sklearn.coef_[0]:.4f}")
print(f"   • Equation: y = {model_sklearn.intercept_:.4f} + {model_sklearn.coef_[0]:.4f}x")

print(f"\n📈 INTERPRETATION:")
print(f"   • For every 1 unit increase in X, y increases by {model_sklearn.coef_[0]:.4f} units")
print(f"   • When X = 0, the predicted value of y is {model_sklearn.intercept_:.4f}")
print(f"   • The model explains {r2_score(y_test, y_test_pred)*100:.1f}% of the variance in y")

print(f"\n✅ NEXT STEPS:")
print(f"   • Verify all assumptions are met")
print(f"   • Consider polynomial features if non-linearity exists")
print(f"   • Try regularized regression (Ridge/Lasso) for multiple features")
print(f"   • Collect more data if performance is insufficient")

print("\n" + "=" * 50)

## 🚀 10. Exercises

### Exercise 1: Multiple Linear Regression
- Generate a dataset with 3 features
- Fit a multiple linear regression model
- Analyze feature importance

### Exercise 2: Polynomial Regression
- Create non-linear data
- Compare linear vs polynomial regression
- Find optimal polynomial degree

### Exercise 3: Regularized Regression
- Implement Ridge and Lasso regression
- Compare with regular linear regression
- Tune regularization parameter

### Exercise 4: Real Dataset
- Load Boston Housing or California Housing dataset
- Perform complete linear regression analysis
- Report insights and recommendations